In [14]:
# Imports

import gspread as gs
import pandas as pd
import plotly.graph_objects as go

In [15]:
# Match Results Google Sheet
sheet_id = "1RBWOGn_cBeK9qQmSbVz7NE_XGTBwevMVOFWbW9EHGf8"
sheet_name = "Match_Results_2"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
results_df = pd.read_csv(url).fillna('')
results_df = results_df.sort_values('Match Week', ascending=False)
results_df.head(50)

,Timestamp,Match Week,Match Winner,Match Loser,Games Won By Loser,player1,player2,player1score,player2score
55,9/19/2021 2:31:21,Week 3 (9/20),Paul Czeresko,Tiffany Schleigh,0.0,Paul Czeresko,Tiffany Schleigh,1,0
57,9/19/2021 17:58:38,Week 2 (9/13),Adam Fratino,George Brown,1.0,Adam Fratino,George Brown,1,0
50,9/16/2021 21:14:16,Week 2 (9/13),Nick Shields,Sarah Mroue,0.0,Nick Shields,Sarah Mroue,1,0
45,9/12/2021 15:13:33,Week 2 (9/13),Eric Papa,Steve Olsen,0.0,Eric Papa,Steve Olsen,1,0
46,9/14/2021 19:11:45,Week 2 (9/13),Mike Dunlap,Paul Czeresko,1.0,Mike Dunlap,Paul Czeresko,1,0
47,9/15/2021 19:24:05,Week 2 (9/13),Missy Kayko,Paul Jones,0.0,Missy Kayko,Paul Jones,1,0
48,9/15/2021 21:13:36,Week 2 (9/13),Roger Gibian,Tiffany Schleigh,0.0,Roger Gibian,Tiffany Schleigh,1,0
49,9/16/2021 20:37:59,Week 2 (9/13),Matt Bird,Deb Czeresko,0.0,Matt Bird,Deb Czeresko,1,0
51,9/16/2021 23:44:52,Week 2 (9/13),Ben Cole,Dana Vandagriff,0.0,Ben Cole,Dana Vandagriff,1,0
52,9/17/2021 10:24:33,Week 2 (9/13),Nelson Bakerman,Ian Ainley,1.0,Nelson Bakerman,Ian Ainley,1,0


In [16]:
# Creating Won_Loss Results Table
winners = dict(enumerate(results_df['Match Winner']))
losers = dict(enumerate(results_df['Match Loser']))
winners.update(losers)
players = winners
wins = results_df['player1'].value_counts()
losses = results_df['player2'].value_counts()
w_l = pd.concat([wins,losses],axis=1).fillna(0).astype(int).round().rename(columns={'player1':'Wins','player2':'Losses'})
w_l.loc[:,'Matches']=w_l.sum(numeric_only=True, axis=1)
w_l['Win%'] = w_l['Wins']/w_l['Matches']
w_l['Player']=w_l.index
column_names = ['Player',"Matches", "Wins", "Losses", "Win%"]
w_l['Win%'] = w_l['Win%'].round(3)
w_l = w_l.reindex(columns=column_names)
w_l = w_l.reset_index(drop=True)
w_l.head(10)

,Player,Matches,Wins,Losses,Win%
0,Mike Dunlap,7,6,1,0.857
1,Missy Kayko,7,6,1,0.857
2,Ben Brown,5,5,0,1.000
3,Jeff Ziev,6,4,2,0.667
4,Eric Papa,5,4,1,0.800
5,Ryan Leggette,4,3,1,0.750
6,Nelson Bakerman,4,3,1,0.750
7,Jeff Weber,3,2,1,0.667
8,Ian Ainley,4,2,2,0.500
9,Justin Goodfellow,3,2,1,0.667


In [17]:
# Rankings
rankings_df = pd.read_csv('Rankings.csv')
rankings_df.drop_duplicates(subset=['Player'],inplace=True)
rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.loc[:,'ELO']=1600
initial_ranking = 1600
k_factor = 32
rankings_df.head(20)

,Player,ELO
Player,,
Ben Brown,Ben Brown,1600
Missy Kayko,Missy Kayko,1600
Mike Dunlap,Mike Dunlap,1600
Eric Papa,Eric Papa,1600
Jeff Ziev,Jeff Ziev,1600
Ryan Leggette,Ryan Leggette,1600
Paul Assad,Paul Assad,1600
Jeff Weber,Jeff Weber,1600
Duan Knibbs,Duan Knibbs,1600


In [18]:
# ELO Formula
def elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor):
    R1 = 10**(p1_ranking/400)
    R2 = 10**(p2_ranking/400)
    exp_1 = R1/(R1+R2)
    exp_2 = R2/(R1+R2)
    ELO_1 = p1_ranking+k_factor*(p1_score-exp_1)
    ELO_2 = p2_ranking+k_factor*(p2_score-exp_2)
    return ELO_1, ELO_2

In [19]:
#this loop reads the scores in order starting from the top of the matches
#every time you run this loop it will read all of the scores again
for idx in results_df.index:
    player_1 = results_df.player1[idx]
    p1_score = results_df.player1score[idx]
    player_2 = results_df.player2[idx]
    p2_score = results_df.player2score[idx]
    current_rankings_list = list(set(rankings_df.index))
    if player_1 in current_rankings_list:
        p1_ranking = rankings_df.at[player_1,'ELO']
    else:
        p1_ranking = initial_ranking
    if player_2 in current_rankings_list:
        p2_ranking = rankings_df.at[player_2,'ELO']
    else:
        p2_ranking = initial_ranking
    
    ELO_1, ELO_2 = elo_formula(p1_ranking,p2_ranking,p1_score,p2_score,k_factor)
    
    if player_1 in current_rankings_list:
        rankings_df.at[player_1,'ELO'] = ELO_1
    else:
        new_ELO_dict = {'Player':player_1,'ELO':ELO_1}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
    if player_2 in current_rankings_list:
        rankings_df.at[player_2,'ELO'] = ELO_2
    else:
        new_ELO_dict = {'Player':player_2,'ELO':ELO_2}
        rankings_df = rankings_df.append(new_ELO_dict, ignore_index=True)
        rankings_df.set_index(['Player'],inplace=True,drop=False)
rankings_df.reset_index(drop=True,inplace=True)
rankings_df.sort_values(by=['ELO'],inplace=True,ascending=False,ignore_index=True)
rankings_df.index+=1
rankings_df.to_csv('Rankings.csv',index=False)

In [20]:
pd.set_option('display.max_rows', None)
rankings_df

,Player,ELO
1,Ben Brown,1679.512574
2,Mike Dunlap,1673.427068
3,Missy Kayko,1672.318835
4,Eric Papa,1644.216138
5,Jeff Ziev,1632.505637
6,Nelson Bakerman,1629.652436
7,Ryan Leggette,1629.618756
8,Paul Assad,1618.593569
9,Jeff Weber,1617.504940
10,Justin Goodfellow,1616.059821


In [21]:
# Formatting and Printing Final Table
final_table = w_l.merge(rankings_df, how="left", on="Player")
final_table_print = final_table.drop('Matches', axis=1)
final_table_print['Ranking'] = final_table_print[["ELO","Win%"]].apply(tuple,axis=1)\
             .rank(ascending=False, method="min").astype(int)
final_table_print['Ranking'] = final_table_print['Ranking'].round(3)
final_table_print['ELO'] = final_table_print['ELO'].round(0)
column_names = ['Ranking','Player', "ELO", "Wins", "Losses", "Win%"]
final_table_print = final_table_print.reindex(columns=column_names)
final_table_print = final_table_print.sort_values(by=['Ranking','Win%'],ascending=[True,False])
final_table_print


,Ranking,Player,ELO,Wins,Losses,Win%
2,1,Ben Brown,1680.0,5,0,1.000
0,2,Mike Dunlap,1673.0,6,1,0.857
1,3,Missy Kayko,1672.0,6,1,0.857
4,4,Eric Papa,1644.0,4,1,0.800
3,5,Jeff Ziev,1633.0,4,2,0.667
6,6,Nelson Bakerman,1630.0,3,1,0.750
5,7,Ryan Leggette,1630.0,3,1,0.750
11,8,Paul Assad,1619.0,2,1,0.667
7,9,Jeff Weber,1618.0,2,1,0.667
9,10,Justin Goodfellow,1616.0,2,1,0.667


In [22]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=list(column_names),
                fill_color='darkblue',
                font_color='white',
                font_size=16,
                align='center'),
    cells=dict(values=[
                final_table_print.Ranking,
                final_table_print.Player,
                final_table_print.ELO,
                final_table_print.Wins,
                final_table_print.Losses, 
                final_table_print['Win%']
                ],
               fill_color='white',
               font_color='black',
               font_size=14,
               align='center',
               height=30,
               line_color='lightgray',
               format=["1234","","#","#","#",".1%"]
               ),
    columnwidth=20)])

fig.show()
fig.write_html("final_table.html", include_plotlyjs="cdn")

final_table_print.to_csv('Rankings_Table.csv',index=False)
final_table_print.to_excel('Rankings_Table.xlsx',index=False)